### 对分类模型进行量化，并测试精度

In [7]:
import torchvision
from ppq import *
from ppq.api import *
from Utilities.Imagenet import (evaluate_mmlab_module_with_imagenet,
                                evaluate_onnx_module_with_imagenet,
                                evaluate_ppq_module_with_imagenet,
                                evaluate_torch_module_with_imagenet,
                                load_imagenet_from_directory)
import os

### 测试全精度模型

In [8]:
CFG_PLATFORM = TargetPlatform.TRT_INT8  # 用来指定目标平台
platform = "TRT"  #记得修改上面两个
QUANT_SETTING = QuantizationSettingFactory.trt_setting() # 用来指定量化配置

CFG_DEVICE = 'cuda'                            # 一个神奇的字符串，用来确定执行设备
CFG_BATCHSIZE = 64                             # 测试与calib时的 batchsize
CFG_INPUT_SHAPE = (CFG_BATCHSIZE, 3, 224, 224) # 用来确定模型输入的尺寸，好像 imagenet 都是这个尺寸
CFG_VALIDATION_DIR = '/home/geng/tinyml/ppq/benchmark/Assets/Imagenet_Valid'   # 用来读取 validation dataset
CFG_TRAIN_DIR = '/home/geng/tinyml/ppq/benchmark/Assets/Imagenet_Train'        # 用来读取 train dataset，注意该集合将被用来 calibrate 你的模型
CFG_DUMP_PATH = '/home/geng/tinyml/ppq/benchmark/classification/'+platform+'_output'    # 所有模型保存的路径名

if not os.path.exists(CFG_DUMP_PATH):
    os.makedirs(CFG_DUMP_PATH)

|model|TargetPlatform|ORT FP32|PPQ INT8|DQD ORT INT8|RealPlatform INT8|
|----|----|----|----|----|----|
|resnet18|OpenVino|69.764|69.466|67.109|-|
|resnet18|TRT|69.764|69.548|69.524|-|
|resnet18|Snpe|69.764|69.278|69.266|-|
|resnet18|Ncnn|69.764|69.106|69.070|-|

In [5]:
with ENABLE_CUDA_KERNEL():
    model_builder, model_name  = torchvision.models.resnet18, 'resnet18'

    print(f'---------------------- PPQ Quantization Test Running with {model_name} ----------------------')
    model = model_builder(pretrained=True).to(CFG_DEVICE)

    #测试FP32精度
    # fp32_report = evaluate_torch_module_with_imagenet(
    #     model=model, imagenet_validation_dir=CFG_VALIDATION_DIR,
    #     batchsize=CFG_BATCHSIZE, device=CFG_DEVICE, verbose=True)

    # 获取校准数据
    dataloader = load_imagenet_from_directory(
        directory=CFG_TRAIN_DIR, batchsize=CFG_BATCHSIZE,
        shuffle=False, subset=5120, require_label=False,
        num_of_workers=8)

    # 量化torch模型
    ppq_quant_ir = quantize_torch_model(
        model=model, calib_dataloader=dataloader, input_shape=CFG_INPUT_SHAPE,
        calib_steps=5120 // CFG_BATCHSIZE, collate_fn=lambda x: x.to(CFG_DEVICE), verbose=1,
        device=CFG_DEVICE, platform=CFG_PLATFORM, setting=QUANT_SETTING,
        onnx_export_file=f'{os.path.join(CFG_DUMP_PATH, model_name)}-FP32.onnx')
        
    # 评估PPQ量化后的模型
    ppq_int8_report = evaluate_ppq_module_with_imagenet(
        model=ppq_quant_ir, imagenet_validation_dir=CFG_VALIDATION_DIR,
        batchsize=CFG_BATCHSIZE, device=CFG_DEVICE, verbose=True)

    # 导出ORT模型
    export_ppq_graph(
        graph=ppq_quant_ir, 
        platform=TargetPlatform.ONNXRUNTIME,
        graph_save_to=f'{os.path.join(CFG_DUMP_PATH, model_name)}-INT8.onnx')
    
    # 导出部署平台模型
    export_ppq_graph(
        graph=ppq_quant_ir, 
        platform=TargetPlatform.TRT_INT8,
        graph_save_to=f'{os.path.join(CFG_DUMP_PATH, model_name)}-INT8.onnx')
    
    # 评估onnx运行模型
    evaluate_onnx_module_with_imagenet(
        onnxruntime_model_path=f'{os.path.join(CFG_DUMP_PATH, model_name)}-INT8.onnx', 
        imagenet_validation_dir=CFG_VALIDATION_DIR, batchsize=CFG_BATCHSIZE, 
        device=CFG_DEVICE)

    # ppq_int8_report.to_csv(f'{os.path.join(CFG_DUMP_PATH, model_name)}-report.csv')


PPQ is compling CUDA Kernels. Please wait...If there is any problem with kernel compilation, feel free to remove ENABLE_CUDA_KERNEL clause.
---------------------- PPQ Quantization Test Running with resnet18 ----------------------


/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Evaluating Model...:   0%|          | 0/781 [00:00<?, ?it/s]/home/geng/tinyml/ppq/benchmark/classification/Utilities/Imagenet/imagenet_util.py:188: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prec1

Test: [0 / 781]	Prec@1 84.375 (84.375)	Prec@5 95.312 (95.312)


Evaluating Model...:  13%|█▎        | 104/781 [00:05<00:31, 21.38it/s]

Test: [100 / 781]	Prec@1 76.269 (76.269)	Prec@5 92.280 (92.280)


...

Test: [600 / 781]	Prec@1 70.255 (70.255)	Prec@5 89.588 (89.588)


Evaluating Model...:  90%|████████▉ | 701/781 [08:25<02:09,  1.62s/it]

Test: [700 / 781]	Prec@1 69.307 (69.307)	Prec@5 88.844 (88.844)


Evaluating Model...: 100%|██████████| 781/781 [10:34<00:00,  1.23it/s]


 * Prec@1 69.266 Prec@5 88.896


## 评估Openvino推理精度

In [1]:
import openvino
import openvino.runtime
import cfg
import torch
from ppq.core import *
from Utilities.Imagenet import evaluate_openvino_module_with_imagenet

CFG_BATCHSIZE = 64                             # 测试与calib时的 batchsize
CFG_INPUT_SHAPE = (CFG_BATCHSIZE, 3, 224, 224) # 用来确定模型输入的尺寸，好像 imagenet 都是这个尺寸
CFG_VALIDATION_DIR = '/home/geng/tinyml/ppq/benchmark/Assets/Imagenet_Valid'   # 用来读取 validation dataset
CFG_TRAIN_DIR = '/home/geng/tinyml/ppq/benchmark/Assets/Imagenet_Train'        # 用来读取 train dataset，注意该集合将被用来 calibrate 你的模型



      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/


TensorRT is not installed, TRT Exporter is disabled.


In [2]:
openvino_executor = openvino.runtime.Core()
name = 'ResNet18'
target = "OpenVino"

model_int8 = f'{os.path.join(cfg.OPENVINO_BASE_PATH, name)}-{target}-INT8.onnx'

model_openvino = openvino_executor.compile_model(
    model = openvino_executor.read_model(model=model_int8), device_name="CPU")
evaluate_openvino_module_with_imagenet(model=model_openvino,imagenet_validation_dir=CFG_VALIDATION_DIR,
batchsize=CFG_BATCHSIZE,device="cpu")

Evaluating Model...:   0%|          | 0/781 [00:00<?, ?it/s]/home/geng/tinyml/ppq/benchmark/classification/Utilities/Imagenet/imagenet_util.py:208: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prec1, prec5 = accuracy(torch.tensor(batch_pred).to('cpu'), batch_label.to('cpu'), topk=(1, 5))
Evaluating Model...:   0%|          | 1/781 [00:01<13:10,  1.01s/it]

Test: [0 / 781]	Prec@1 81.250 (81.250)	Prec@5 92.188 (92.188)


Evaluating Model...:  13%|█▎        | 101/781 [00:32<03:33,  3.18it/s]

Test: [100 / 781]	Prec@1 74.165 (74.165)	Prec@5 91.352 (91.352)


Evaluating Model...:  26%|██▌       | 201/781 [01:03<03:02,  3.18it/s]

...

Evaluating Model...:  90%|████████▉ | 701/781 [03:39<00:24,  3.22it/s]

Test: [700 / 781]	Prec@1 67.196 (67.196)	Prec@5 87.230 (87.230)


Evaluating Model...: 100%|██████████| 781/781 [04:03<00:00,  3.21it/s]

 * Prec@1 66.985 Prec@5 87.090


,loss,top1_accuracy,top5_accuracy,batch_time
0,tensor(0.8667),81.2500,92.1875,0.397991
1,tensor(0.6025),82.8125,96.8750,0.346456
2,tensor(1.0097),71.8750,90.6250,0.325113
3,tensor(0.7397),81.2500,96.8750,0.313082
4,tensor(1.3620),70.3125,87.5000,0.301671
...,...,...,...,...
776,tensor(0.4755),87.5000,95.3125,0.263489
777,tensor(0.7493),85.9375,93.7500,0.264261
778,tensor(0.6864),81.2500,95.3125,0.262163
779,tensor(1.8074),53.1250,79.6875,0.263277


In [4]:

INPUT_SHAPE = (64, 3, 224, 224) # 用来确定模型输入的尺寸，好像 imagenet 都是这个尺寸
SAMPLES = torch.rand(size=INPUT_SHAPE)
res = torch.tensor(list(model_openvino([convert_any_to_numpy(SAMPLES)]).values())[0])   #openvino获取结推理果

In [7]:
torch.tensor(list(res)[0])

tensor([[-2.0925,  0.9155,  1.4386,  ..., -1.0462,  0.1308,  2.4848],
        [-1.7001,  0.7847,  1.5694,  ..., -0.9155,  0.0000,  2.2233],
        [-1.7001,  0.6539,  1.1770,  ..., -0.9155,  0.1308,  2.2233],
        ...,
        [-1.5694,  0.6539,  1.3078,  ..., -0.7847,  0.1308,  2.6156],
        [-1.7001,  0.7847,  1.4386,  ..., -0.7847,  0.0000,  2.6156],
        [-2.0925,  0.5231,  1.1770,  ..., -0.9155,  0.2616,  2.4848]])